# Recurrent Neural Networks
In this tutorial we'll learn the basics about sequence modeling using deep recurrent models.

## Time matters
In the past tutorials, we learnt how to model time-agnostic data with deep neural networks. For many problems this is not enough. As an example, assume you want to predict the next word in a sentence given the sequence of words up to it. It is very likely that such prediction depends, in some way, on the order such previous words are presented.
Recurrent Networks (RNNs) are designed to perform some task for every element of a sequence, and to keep a **state** (memory) that stores information about previous computations. 
A very common way to picture RNNs is the following:

<img src="img/rnn.jpg" width="600">

The image shows how a RNN can be unrolled into a full network, meaning that we can write it for the complete sequence. For example, if the sequence we fit is 5 timestamps long, the RNN can be unrolled into a regular feed forward neural network with 5 layers, one for each sequence element. The usual backpropagation algorithm can be applied in this context.

**We just need some math now**.

* With $x_t$ we denote the input at time step $t$. For example, it could be a one hot vector identifying a certain word in a dictionary;
* $s_t$ denotes the hidden state at time step $t$, and encodes the "memory". $s_t$ is dependent on the previous hidden state and on the current input: $s_t=\phi(Ux_t + Ws_{t-1})$. The function $f$ is a non-linearity such as ReLU or tanh. 
* $o_t$ denotes the output at a step $t$. It depends only on the current state: $o_t = \phi(Vs_t)$

### Drawbacks
There are mainly two issues when training a RNN. 
The first one is the following. As the length of the sequence grows, the "unrolled" version of the net becomes very deep, and suffers **exploding or vanishing gradients**.
Moreover, RNNs are capable of modeling the state in arbitrarily long sequences just in theory. When it comes to reality, they just look a **few steps** back in the past.

## Long short term memory
To overcome the issues affecting recurrent networks, usually a gated unit is used. One famous example is the Long Short Term Memory (LSTM) model. This model defines a special kind of RNN, capable of learning long-term dependencies. It works tremendously well on a large variety of problems, and are now widely used.
Their behavior is defined by a cell storing the state, and some gates allowing or denying some types of access to such cell:


<img src="img/LSTM.png" width="500">

Basically, this unit learns to build signals such as $i_t$, $o_t$, $f_t$ $\in (0,1)$, and adjust the processes of writing, reading and resetting the information the cell holds through the following math:
* $i_t = \sigma(\theta_{xi}X_t + \theta_{hi}h_{t-1} + b_i)$
* $f_t = \sigma(\theta_{xf}X_t + \theta_{hf}h_{t-1} + b_f)$
* $o_t = \sigma(\theta_{xo}X_t + \theta_{ho}h_{t-1} + b_o)$
* $g_t = tanh(\theta_{xg}X_t + \theta_{hg}h_{t-1} + b_g)$
* $c_t = f_t \otimes c_{t-1}+i_t \otimes g_t$
* $h_t = o_t \otimes tanh(c_t)$

where $\otimes$ represents element-wise multiplication.

# Let's start practice
Today's task is called **sentiment analysis**. We will try to classify tweets into two categories: `positive` vs `negative`.

As this task involves natural language processing, we need to model the sequence of words of each tweet.

To this purpose, we provide two different files, namely `data/tweets_train.csv` and `data/tweets_train.csv`. Each file contains a list of tweets along with their corresponding label. In order to encode a word into a numerical vector, we need to do the following:
* create a *dictionary* holding all the words contained in the training data. It is common to clip this structure to a fixed dimension (e.g. keeping only the $N$ most common words);
* represent each word as a $N$ dimensional one-hot vector which encodes its position in the *dictionary*.

In [1]:
from __future__ import print_function
import numpy as np
from keras.models import Model
from keras.layers import LSTM, Input, Dense, Flatten, MaxPooling1D
import re
from collections import Counter
import csv


# each tweet is made by max. 140 characters
MAX_TWEET_CHARS = 140


def preprocess(line):
    """
    Pre-process a string of text. Eventually add additional pre-processing here.
    """
    line = line.lower()               # turn to lowercase
    line = line.replace('\n', '')     # remove newlines
    line = re.sub(r'\W+', ' ', line)  # keep characters only (\W is short for [^\w])

    return line


def get_dictionary(filename, dict_size=2000):
    """
    Read the tweets and return a list of the 'max_words' most common words.
    """
    all_words = []
    with open(filename, 'rb') as csvfile:
        r = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in r:
            tweet = row[3]
            if len(tweet) <= MAX_TWEET_CHARS:
                words = preprocess(tweet).split()
                all_words += words

    word_counter = Counter(all_words)
    dictionary, _ = zip(*word_counter.most_common(min(dict_size, len(word_counter))))

    return dictionary

Using Theano backend.
Using gpu device 0: Quadro K2200 (CNMeM is disabled, cuDNN 5005)


So, our training data will have size `(n_examples, sequence_length, N)`.

Since the size of such structure may be very large, we use a **generator** to load data batches only when needed during the training stage.

In [2]:
def generate_batch(filename, batchsize, maxlen=50, dict_size=2000):
    """
    Generate a batch of training data
    """

    # get the list of words that will constitute our dictionary (once only)
    dictionary = get_dictionary(filename, dict_size)

    # read training data (once only)
    rows = []
    with open(filename, 'rb') as csvfile:
        reader = csv.reader(csvfile, delimiter=',', quotechar='"')
        for row in reader:
            rows.append(row)

    while True:

        # prepare data structures
        X_batch = np.zeros((batchsize, maxlen, len(dictionary)+1), dtype=np.float32)
        Y_batch = np.zeros(batchsize, dtype=np.float32)

        for i in range(batchsize):

            rand_idx = np.random.randint(0, len(rows))
            Y_batch[i] = float(rows[rand_idx][1])

            random_tweet = rows[rand_idx][3]
            if len(random_tweet) <= MAX_TWEET_CHARS:

                words = preprocess(random_tweet).split()

                # vectorization
                for j, w in enumerate(words):
                    try:
                        w_idx = dictionary.index(w)
                        X_batch[i, j, w_idx + 1] = 1
                    except Exception:
                        # word not found, using the unknown
                        X_batch[i, j, 0] = 1
            else:
                i -= 1

        yield X_batch, Y_batch

To speed things up, we already prepared the skeleton of the `__main__` file. Feel free to try different models!

In [5]:
if __name__ == '__main__':

    max_len = 50      # max sequence length
    dict_size = 1500  # dictionary clipping 
    batchsize = 128 

    model_in = Input(shape=(max_len, dict_size + 1))
    # ...
    # your model goes here
    # ...
       
    model_out = Dense(1, activation='sigmoid')(x)

    model = Model(input=model_in, output=model_out)
    model.compile(loss='binary_crossentropy', optimizer='adam')

    model.fit_generator(generate_batch('data/tweets_train.csv', batchsize, maxlen=max_len, dict_size=dict_size),
                        nb_epoch=1,
                        samples_per_epoch=256*batchsize,
                        validation_data=generate_batch('data/tweets_val.csv', batchsize, maxlen=max_len, dict_size=dict_size),
                        nb_val_samples=100*batchsize)

Epoch 1/1
32768/32768 [==============================] - 79s - loss: 0.5337 - val_loss: 0.9518


### Testing the model:
When training ends, you can test the model by classifying your own tweets:

In [ ]:
    print('TEST phase: ' + 60*'-')

    dictionary = get_dictionary('data/tweets_train.csv', dict_size)

    while True:
        my_tweet = raw_input('Write your own tweet (max {} words):\n'.format(max_len)).lower()
        if my_tweet:
            words = preprocess(my_tweet).split()
            X_test = np.zeros((1, max_len, dict_size + 1))
            # vectorization
            for j, w in enumerate(words):
                try:
                    w_idx = dictionary.index(w)
                    X_test[0, j, w_idx + 1] = 1
                except Exception:
                    # word not found, using the unknown
                    X_test[0, j, 0] = 1

            Y_test = model.predict(X_test)
            print('Model output: {}'.format(Y_test))
            print(':)' if Y_test > 0.5 else ':(')
        else:
            break

TEST phase: ------------------------------------------------------------
Write your own tweet (max 50 words):
what a wonderful morning!
Model output: [[ 0.81711435]]
:)
Write your own tweet (max 50 words):
i want to die asap
Model output: [[ 0.0594591]]
:(
